In [8]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans


# Load the adjacency matrix
file_path = '/Users/gianmarcoferrara/Desktop/adjacency_matrices/adjacency_matrix_2.csv'  # Replace with your actual file path
adjacency_matrix = pd.read_csv(file_path, header=None).values


# Create a directed graph from the adjacency matrix
G = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

# Calculate raw out-degree and in-degree
out_degree = {node: G.out_degree(node) for node in G.nodes()}
in_degree = {node: G.in_degree(node) for node in G.nodes()}

# Step 4: Calculate the difference between out-degree and in-degree using dictionary comprehension
degree_differences = {node: out_degree[node] - in_degree[node] for node in out_degree}

print(out_degree)
print(in_degree)

print(degree_differences)



# Convert the dictionary to a DataFrame and save as CSV
degree_differences_df = pd.DataFrame(list(degree_differences.items()), columns=['Node', 'Degree_Difference'])
degree_differences_df.to_csv('degree_differences_2.csv', index=False)

print("Degree differences saved to 'degree_differences.csv'")


{0: 3104, 1: 31, 2: 601, 3: 2, 4: 133, 5: 492, 6: 4, 7: 1, 8: 387, 9: 5, 10: 191, 11: 462, 12: 45, 13: 66, 14: 40, 15: 137, 16: 56, 17: 432, 18: 66, 19: 20, 20: 163, 21: 55, 22: 449, 23: 395, 24: 30, 25: 510, 26: 171, 27: 354, 28: 545, 29: 12, 30: 1, 31: 312, 32: 1, 33: 1, 34: 1, 35: 439, 36: 146, 37: 603, 38: 99, 39: 471, 40: 298, 41: 591, 42: 473, 43: 1, 44: 2, 45: 606, 46: 3, 47: 439, 48: 128, 49: 221, 50: 5, 51: 1, 52: 343, 53: 495, 54: 1, 55: 1, 56: 31, 57: 315, 58: 621, 59: 619, 60: 1, 61: 392, 62: 1, 63: 161, 64: 1, 65: 83, 66: 1, 67: 11, 68: 1, 69: 174, 70: 4, 71: 156, 72: 226, 73: 10, 74: 11, 75: 495, 76: 410, 77: 1, 78: 243, 79: 311, 80: 17, 81: 198, 82: 95, 83: 263, 84: 238, 85: 2, 86: 80, 87: 288, 88: 34, 89: 6, 90: 2, 91: 34, 92: 209, 93: 1, 94: 12, 95: 60, 96: 12, 97: 60, 98: 266, 99: 284, 100: 2, 101: 2, 102: 200, 103: 108, 104: 55, 105: 429, 106: 132, 107: 8, 108: 304, 109: 215, 110: 265, 111: 266, 112: 55, 113: 252, 114: 254, 115: 2, 116: 98, 117: 16, 118: 293, 119: 9,